In [ ]:
### NOTE: this notebook is from https://machinelearningcoban.com/2017/04/09/smv/

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

# Compute distance between each pair of the two collections of inputs
from scipy.spatial.distance import cdist
np.random.seed(1)

In [ ]:
means = [[1, 1], [5, 4]]
cov = [[.5, .3], [.3, .5]]
N = 20
X0 = np.random.multivariate_normal(means[0], cov, N)
X1 = np.random.multivariate_normal(means[1], cov, N)
X = np.vstack((X0, X1)).T
y = np.hstack((np.ones((1, N)), -1*np.ones((1, N))))

### SVM Primal problem:
$$\begin{array}{c}
\min _{w, b} \frac{1}{2}|| w||^{2} \\
y_{i}\left(w^\top x_{i}+b\right) \geq 1\; \forall i
\end{array}$$


### Lagrangian:
$$\mathcal{L}(w, b, \alpha)=\frac{1}{2} \Vert w \Vert ^{2}-\sum_{i} \alpha_{i}\left[y_{i}\left(w^\top x_{i}+b\right)-1\right]$$
where $\alpha_i \geq 0$ is the Lagrangian multiplier.


### Lagrange dual function:
$$g(\alpha) = \sum_i \alpha_i - \frac{1}{2}\sum_{ij} \alpha_i \alpha_j y_i y_j x_i^\top x_j$$
with $\sum_i \alpha_i y_i = 0 \text{,}\space \alpha_i \geq 0$.


Let:

$\mathbf{V}=\left[y_{1} x_{1}, y_{2} x_{2}, \ldots, y_{N} x_{N}\right]$,

$\mathbf{1}=[1,1, \ldots, 1]^\top$,

$\mathbf{K} = \mathbf{V}^\top \mathbf{V}$,

$$\Rightarrow g(\alpha)=-\frac{1}{2} \alpha^\top \mathbf{K} \alpha+\mathbf{1}^\top \alpha$$



In [ ]:
#solving the problem
from cvxopt import matrix, solvers
# build V
# your code is here 
V = 
##########

# your code is here
K = 
##########

p = matrix(-np.ones((2*N, 1))) # all-one vector 
# build A, b, G, h 
G = matrix(-np.eye(2*N)) # for all lambda_n >= 0
h = matrix(np.zeros((2*N, 1)))
A = matrix(y) # the equality constrain is actually y^T lambda = 0
b = matrix(np.zeros((1, 1))) 


solvers.options['show_progress'] = False
sol = solvers.qp(K, p, G, h, A, b)

alpha = np.array(sol['x'])

Support vectors are positive solutions ($\alpha > 0$) of the Lagrange dual function.

$w$ and $b$ can be calculated from support vector set $\mathcal{S}$:

$$w=\sum_{m \in \mathcal{S}} \alpha_{m} y_{m} x_{m}$$

$$b=\frac{1}{N_{\mathcal{S}}} \sum_{n \in \mathcal{S}}\left(y_{n}-w^\top x_{n}\right)$$
where  $N_{\mathcal{S}}$ is the number of support vectors.


In [ ]:
# Finding support vector
epsilon = 1e-5 # just a small number, greater than 1e-9
S = np.where(alpha > epsilon)[0]
alphaS = alpha[S]

In [ ]:
alphaS

In [ ]:
VS = V[:, S]
XS = X[:, S]
yS = y[:, S]
# calculate w and b
### your code is here
w = 
b = 
###########


print('w = ', w.T)
print('b = ', b)

In [ ]:
def plot(w, b, X0, X1, X, S):
  fig, ax = plt.subplots(figsize=(10,10))
  x1 = np.arange(-10, 10, 0.1)
  y1 = -w[0, 0]/w[1, 0]*x1 - b/w[1, 0]
  y2 = -w[0, 0]/w[1, 0]*x1 - (b-1)/w[1, 0]
  y3 = -w[0, 0]/w[1, 0]*x1 - (b+1)/w[1, 0]
  plt.plot(x1, y1, 'k', linewidth = 3)
  plt.plot(x1, y2, 'k')
  plt.plot(x1, y3, 'k')


  # y4 = 10*x1
  # plt.plot(x1, y1, 'k')
  # plt.fill_between(x1, y1, color='red', alpha=0.1)
  # plt.fill_between(x1, y1, y4, color = 'blue', alpha = .1)


  plt.plot(X0[:, 0], X0[:, 1], 'bs', markersize = 8, alpha = .8)
  plt.plot(X1[:, 0], X1[:, 1], 'ro', markersize = 8, alpha = .8)

  plt.axis('equal')
  plt.ylim(np.min(X[1, :])*1.05, np.max(X[1, :])*1.05)
  plt.xlim(np.min(X[0, :])*1.05, np.max(X[0, :])*1.05)

  # hide tikcs 
  cur_axes = plt.gca()
  cur_axes.axes.get_xaxis().set_ticks([])
  cur_axes.axes.get_yaxis().set_ticks([])

  # add circles around support vectors 
  for m in S:
      circle = plt.Circle((X[0, m], X[1, m] ), 0.1, color='k', fill = False)
      ax.add_artist(circle)


  plt.xlabel('$x_1$', fontsize = 20)
  plt.ylabel('$x_2$', fontsize = 20)
  plt.show()
plot(w, b, X0, X1, X, S)